In [1]:
import os 
zipllm_dir = "../HF_storage"          # HF_storage dir
fastcdc_dir = "../experiment/results/fastcdc_chunks"     # FastCDC results dir
zipnn_file = "../experiment/results/zipnn_results.json"  # ZipNN results file
output_file = "./plots/ddr_comparison.png"          # output image file

# Ensure the output directory exists for output_file
os.makedirs(os.path.dirname(output_file), exist_ok=True)

In [46]:
import os
import json
import argparse
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import glob
from typing import Dict, List, Set, Tuple

def process_zipllm_results(storage_dir: str) -> List[Tuple[int, float]]:
    model_metadata_dir = os.path.join(storage_dir, "model_metadata")
    model_files = sorted(os.listdir(model_metadata_dir))
    
    unique_tensors = set()
    total_original_size = 0
    unique_tensors_size = 0
    
    results = []
    
    for i, model_file in enumerate(model_files, 1):
        model_path = os.path.join(model_metadata_dir, model_file)
        with open(model_path, 'r') as f:
            model_data = json.load(f)
            
        model_tensors = set()
        model_size = 0
        
        for file_name, file_hash in model_data["files"].items():
            file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
            
            with open(file_metadata_path, 'r') as f:
                file_data = json.load(f)
            model_size += file_data["size"]
            for tensor_name, tensor_hash in file_data.get("tensor_hashes", {}).items():
                model_tensors.add(tensor_hash)
        
        total_original_size += model_size
        new_tensors = model_tensors - unique_tensors
        unique_tensors.update(new_tensors)
        
        for tensor_hash in new_tensors:
            tensor_metadata_path = os.path.join(storage_dir, "tensor_metadata", f"{tensor_hash}.json")
            try:
                with open(tensor_metadata_path, 'r') as f:
                    tensor_data = json.load(f)
                    
                compressed_metadata_path = os.path.join(storage_dir, "compressed_metadata", f"{tensor_hash}.json")
                with open(compressed_metadata_path, 'r') as f:
                    compressed_data = json.load(f)
                    
                unique_tensors_size += compressed_data["compressed_size"]
            except (FileNotFoundError, json.JSONDecodeError):
                continue
        
        ddr = 1 - (unique_tensors_size / total_original_size)
        results.append((i, ddr))
    
    return results


def process_fastcdc_results(fastcdc_dir: str) -> List[Tuple[int, float]]:
    model_dirs = sorted(os.listdir(fastcdc_dir))
    
    unique_chunks = set()
    total_chunk_size = 0
    unique_chunks_size = 0
    
    results = []
    
    for i, model_dir in enumerate(model_dirs, 1):
        model_path = os.path.join(fastcdc_dir, model_dir)
        chunk_files = glob.glob(os.path.join(model_path, "*.chunk.txt"))
        
        model_chunks = set()
        model_chunk_size = 0
        
        for chunk_file in chunk_files:
            with open(chunk_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 2:
                        chunk_hash, chunk_size = parts[0], int(parts[1])
                        model_chunks.add(chunk_hash)
                        model_chunk_size += chunk_size
        
        total_chunk_size += model_chunk_size
        new_chunks = model_chunks - unique_chunks
        unique_chunks.update(new_chunks)
        
        for chunk_file in chunk_files:
            with open(chunk_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 2:
                        chunk_hash, chunk_size = parts[0], int(parts[1])
                        if chunk_hash in new_chunks:
                            unique_chunks_size += chunk_size
        
        ddr = 1 - (unique_chunks_size / total_chunk_size)
        results.append((i, ddr))
    
    return results


def process_zipnn_results(zipnn_file: str) -> List[Tuple[int, float]]:
    with open(zipnn_file, 'r') as f:
        data = json.load(f)
    
    # Sort models to ensure consistent order
    models = sorted(data.keys())
    
    total_original_size = 0
    total_compressed_size = 0
    
    results = []
    
    for i, model in enumerate(models, 1):
        model_data = data[model]
        
        for file, sizes in model_data.items():
            total_original_size += sizes["original_size"]
            total_compressed_size += sizes["compressed_size"]
        
        # Calculate DDR: saved bytes / original total size
        ddr = 1 - (total_compressed_size / total_original_size)
        results.append((i, ddr))
    
    return results


def process_file_dedup_results(storage_dir: str) -> List[Tuple[int, float]]:
    # Get all models
    model_metadata_dir = os.path.join(storage_dir, "model_metadata")
    model_files = sorted(os.listdir(model_metadata_dir))
    
    # Track unique files and their sizes
    unique_files = set()
    total_original_size = 0
    unique_files_size = 0
    
    results = []
    
    for i, model_file in enumerate(model_files, 1):
        model_path = os.path.join(model_metadata_dir, model_file)
        with open(model_path, 'r') as f:
            model_data = json.load(f)
            
        model_files_set = set()
        model_size = 0
        
        # Get all files for this model
        for file_name, file_hash in model_data["files"].items():
            model_files_set.add(file_hash)
            
            file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
            with open(file_metadata_path, 'r') as f:
                file_data = json.load(f)
            
            # Add file size to model size
            model_size += file_data["size"]
        
        # Add to total original size
        total_original_size += model_size
        
        # Add new unique files
        new_files = model_files_set - unique_files
        unique_files.update(new_files)
        
        # Calculate size of unique files
        for file_hash in new_files:
            file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
            with open(file_metadata_path, 'r') as f:
                file_data = json.load(f)
                
            unique_files_size += file_data["size"]
        
        # Calculate DDR: saved bytes / original total size
        ddr = 1 - (unique_files_size / total_original_size)
        results.append((i, ddr))
    
    return results


def plot_results(zipllm_results, fastcdc_results, zipnn_results, file_dedup_results, output_file):
    plt.rcParams['figure.figsize'] = (25, 12)
    plt.rcParams['xtick.major.width'] = 2
    plt.rcParams['ytick.major.width'] = 2
    plt.rcParams['axes.grid'] = False
    plt.rcParams['grid.linestyle'] = '-'
    plt.rcParams['grid.linewidth'] = 1
    plt.rcParams['grid.color'] = '#e1e1e1'
    plt.rcParams['axes.linewidth'] = 2
    plt.rcParams['ytick.major.size'] = 12
    plt.rcParams['xtick.major.size'] = 12
    plt.rcParams['axes.titlesize'] = 52
    plt.rcParams['axes.labelsize'] = 52
    plt.rcParams['lines.linewidth'] = 4
    plt.rcParams['lines.markersize'] = 30
    plt.rcParams['xtick.labelsize'] = 52
    plt.rcParams['ytick.labelsize'] = 52
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['DejaVu Serif']
    plt.rcParams['font.weight'] = 'normal'
    plt.rcParams['axes.labelweight'] = 'normal'
    plt.rcParams['axes.titleweight'] = 'normal'
    plt.rcParams['axes.grid.axis'] = 'both'
    plt.rcParams['axes.grid.which'] = 'major'
    plt.rcParams['figure.dpi'] = 600
    
    plt.figure()
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    if file_dedup_results:
        x_file, y_file = zip(*file_dedup_results)
        line1, = plt.plot(x_file, y_file, color=colors[0], marker='', label='File Dedup')
        plt.text(x_file[-1] + 0.1, y_file[-1], f'{y_file[-1]:.2f}', fontsize=40, va='center')
    
    if fastcdc_results:
        x_fastcdc, y_fastcdc = zip(*fastcdc_results)
        line2, = plt.plot(x_fastcdc, y_fastcdc, color=colors[1], marker='', label='HF(FastCDC)')
        plt.text(x_fastcdc[-1] + 0.1, y_fastcdc[-1], f'{y_fastcdc[-1]:.2f}', fontsize=40, va='center')
    
    if zipnn_results:
        x_zipnn, y_zipnn = zip(*zipnn_results)
        line3, = plt.plot(x_zipnn, y_zipnn, color=colors[2], marker='', label='ZipNN')
        plt.text(x_zipnn[-1] + 0.1, y_zipnn[-1], f'{y_zipnn[-1]:.2f}', fontsize=40, va='center')
    
    if zipllm_results:
        x_zipllm, y_zipllm = zip(*zipllm_results)
        line4, = plt.plot(x_zipllm, y_zipllm, color=colors[3], marker='', label='ZipLLM')
        plt.text(x_zipllm[-1] + 0.1, y_zipllm[-1], f'{y_zipllm[-1]:.2f}', fontsize=40, va='center')
    
    # Add 50% reduction reference line
    plt.axhline(y=0.5, color='gray', linestyle='--', linewidth=3)
    plt.text(1, 0.45, '50% Reduction', fontsize=35, va='bottom',color='gray')
    
    plt.xlabel('Model Count')
    plt.ylabel('Data Reduction Ratio')
    
    plt.ylim(0, 1)
    max_models = max(
        len(zipllm_results) if zipllm_results else 0,
        len(fastcdc_results) if fastcdc_results else 0,
        len(zipnn_results) if zipnn_results else 0,
        len(file_dedup_results) if file_dedup_results else 0
    )
    plt.xticks(range(1, max_models + 1))
    
    plt.legend(loc='upper left', fontsize=40)
    
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(output_file)
    print(f"Plot saved to {output_file}")

In [47]:
zipllm_results = process_zipllm_results(zipllm_dir)
fastcdc_results = process_fastcdc_results(fastcdc_dir)  
zipnn_results = process_zipnn_results(zipnn_file)
file_dedup_results = process_file_dedup_results(zipllm_dir)
plot_results(zipllm_results, fastcdc_results, zipnn_results, file_dedup_results, output_file)

FileNotFoundError: [Errno 2] No such file or directory: '../experiment/results/fastcdc_chunks'

In [ ]:
import os
import json
import glob
import hashlib
import numpy as np
from collections import defaultdict
from typing import Dict, Any

def analyze_fastcdc_chunks(fastcdc_dir: str) -> Dict[str, Any]:
    print("\n===== FastCDC Chunk Deduplication Statistics =====")
    model_dirs = sorted(os.listdir(fastcdc_dir))
    all_chunks = []
    unique_chunks = set()
    chunk_sizes = {}
    for model_dir in model_dirs:
        model_path = os.path.join(fastcdc_dir, model_dir)
        chunk_files = glob.glob(os.path.join(model_path, "*.chunk.txt"))
        for chunk_file in chunk_files:
            with open(chunk_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 2:
                        chunk_hash, chunk_size = parts[0], int(parts[1])
                        all_chunks.append(chunk_hash)
                        unique_chunks.add(chunk_hash)
                        chunk_sizes[chunk_hash] = chunk_size
    total_chunks = len(all_chunks)
    unique_chunk_count = len(unique_chunks)
    chunk_size_values = list(chunk_sizes.values())
    max_chunk_size = max(chunk_size_values) if chunk_size_values else 0
    avg_chunk_size = sum(chunk_size_values) / len(chunk_size_values) if chunk_size_values else 0
    print(f"Total chunks: {total_chunks}")
    print(f"Unique chunks: {unique_chunk_count}")
    print(f"Unique ratio: {unique_chunk_count / total_chunks:.4f}")
    print(f"Max chunk size: {max_chunk_size} bytes ({max_chunk_size / 1024 / 1024:.2f} MB)")
    print(f"Average chunk size: {avg_chunk_size:.2f} bytes ({avg_chunk_size / 1024 / 1024:.2f} MB)")
    return {
        "total_chunks": total_chunks,
        "unique_chunks": unique_chunk_count,
        "unique_ratio": unique_chunk_count / total_chunks if total_chunks else 0,
        "max_chunk_size": max_chunk_size,
        "avg_chunk_size": avg_chunk_size
    }

def analyze_tensor_dedup(storage_dir: str) -> Dict[str, Any]:
    print("\n===== Tensor Deduplication Statistics =====")
    model_metadata_dir = os.path.join(storage_dir, "model_metadata")
    model_files = sorted(os.listdir(model_metadata_dir))
    all_tensors = []
    unique_tensors = set()
    tensor_sizes = {}
    for model_file in model_files:
        model_path = os.path.join(model_metadata_dir, model_file)
        with open(model_path, 'r') as f:
            model_data = json.load(f)
        for file_name, file_hash in model_data["files"].items():
            file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
            try:
                with open(file_metadata_path, 'r') as f:
                    file_data = json.load(f)
                for tensor_name, tensor_hash in file_data.get("tensor_hashes", {}).items():
                    all_tensors.append(tensor_hash)
                    unique_tensors.add(tensor_hash)
                    if tensor_hash not in tensor_sizes:
                        tensor_metadata_path = os.path.join(storage_dir, "tensor_metadata", f"{tensor_hash}.json")
                        try:
                            with open(tensor_metadata_path, 'r') as f:
                                tensor_data = json.load(f)
                                tensor_sizes[tensor_hash] = tensor_data.get("original_size", 0)
                        except (FileNotFoundError, json.JSONDecodeError):
                            tensor_sizes[tensor_hash] = 0
            except (FileNotFoundError, json.JSONDecodeError):
                continue
    total_tensors = len(all_tensors)
    unique_tensor_count = len(unique_tensors)
    tensor_size_values = list(tensor_sizes.values())
    max_tensor_size = max(tensor_size_values) if tensor_size_values else 0
    avg_tensor_size = sum(tensor_size_values) / len(tensor_size_values) if tensor_size_values else 0
    print(f"Total tensors: {total_tensors}")
    print(f"Unique tensors: {unique_tensor_count}")
    print(f"Unique ratio: {unique_tensor_count / total_tensors:.4f}")
    print(f"Max tensor size: {max_tensor_size} bytes ({max_tensor_size / 1024 / 1024:.2f} MB)")
    print(f"Average tensor size: {avg_tensor_size:.2f} bytes ({avg_tensor_size / 1024 / 1024:.2f} MB)")
    return {
        "total_tensors": total_tensors,
        "unique_tensors": unique_tensor_count,
        "unique_ratio": unique_tensor_count / total_tensors if total_tensors else 0,
        "max_tensor_size": max_tensor_size,
        "avg_tensor_size": avg_tensor_size
    }

def analyze_file_dedup(storage_dir: str) -> Dict[str, Any]:
    print("\n===== File Deduplication Statistics =====")
    model_metadata_dir = os.path.join(storage_dir, "model_metadata")
    model_files = sorted(os.listdir(model_metadata_dir))
    all_files = []
    unique_files = set()
    file_sizes = {}
    for model_file in model_files:
        model_path = os.path.join(model_metadata_dir, model_file)
        with open(model_path, 'r') as f:
            model_data = json.load(f)
        for file_name, file_hash in model_data["files"].items():
            all_files.append(file_hash)
            unique_files.add(file_hash)
            if file_hash not in file_sizes:
                file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
                try:
                    with open(file_metadata_path, 'r') as f:
                        file_data = json.load(f)
                        file_sizes[file_hash] = file_data.get("size", 0)
                except (FileNotFoundError, json.JSONDecodeError):
                    file_sizes[file_hash] = 0

    total_files = len(all_files)
    unique_file_count = len(unique_files)
    file_size_values = list(file_sizes.values())
    max_file_size = max(file_size_values) if file_size_values else 0
    avg_file_size = sum(file_size_values) / len(file_size_values) if file_size_values else 0
    total_size_all_files = sum(file_size_values)

    print(f"Total files: {total_files}")
    print(f"Unique files: {unique_file_count}")
    print(f"Unique ratio: {unique_file_count / total_files:.4f}")
    print(f"Max file size: {max_file_size} bytes ({max_file_size / 1024 / 1024:.2f} MB)")
    print(f"Average file size: {avg_file_size:.2f} bytes ({avg_file_size / 1024 / 1024:.2f} MB)")
    print(f"Total size of all files (Model Size): {total_size_all_files} bytes ({total_size_all_files / 1024 / 1024 / 1024:.2f} GB)")

    return {
        "total_files": total_files,
        "unique_files": unique_file_count,
        "unique_ratio": unique_file_count / total_files if total_files else 0,
        "max_file_size": max_file_size,
        "avg_file_size": avg_file_size,
        "total_size_all_files": total_size_all_files
    }


def analyze_layer_dedup(storage_dir: str) -> Dict[str, Any]:
    print("\n===== Layer Deduplication Statistics =====")
    model_metadata_dir = os.path.join(storage_dir, "model_metadata")
    model_files = sorted(os.listdir(model_metadata_dir))
    all_models_layers = []
    tensor_sizes = {}
    for model_file in model_files:
        model_path = os.path.join(model_metadata_dir, model_file)
        with open(model_path, 'r') as f:
            model_data = json.load(f)
        model_layers = defaultdict(dict)
        non_layer_tensors = {}
        for file_name, file_hash in model_data["files"].items():
            file_metadata_path = os.path.join(storage_dir, "file_metadata", f"{file_hash}.json")
            try:
                with open(file_metadata_path, 'r') as f:
                    file_data = json.load(f)
                for tensor_name, tensor_hash in file_data.get("tensor_hashes", {}).items():
                    if tensor_hash not in tensor_sizes:
                        tensor_metadata_path = os.path.join(storage_dir, "tensor_metadata", f"{tensor_hash}.json")
                        try:
                            with open(tensor_metadata_path, 'r') as f:
                                tensor_data = json.load(f)
                                tensor_sizes[tensor_hash] = tensor_data.get("original_size", 0)
                        except (FileNotFoundError, json.JSONDecodeError):
                            tensor_sizes[tensor_hash] = 0
                    parts = tensor_name.split('.')
                    if len(parts) >= 3 and parts[0] == 'model' and parts[1] == 'layers' and parts[2].isdigit():
                        layer_id = int(parts[2])
                        tensor_type = '.'.join(parts[3:])
                        model_layers[layer_id][tensor_type] = tensor_hash
                    else:
                        non_layer_tensors[tensor_name] = tensor_hash
            except (FileNotFoundError, json.JSONDecodeError):
                continue
        all_models_layers.append((model_layers, non_layer_tensors))
    unique_layer_structures = {}
    for model_layers, non_layer_tensors in all_models_layers:
        for layer_id, tensor_dict in model_layers.items():
            tensor_types = sorted(tensor_dict.keys())
            layer_structure = []
            for tensor_type in tensor_types:
                tensor_hash = tensor_dict[tensor_type]
                layer_structure.append(f"{tensor_type}:{tensor_hash}")
            layer_structure_str = "|".join(layer_structure)
            layer_hash = hashlib.sha256(layer_structure_str.encode()).hexdigest()
            layer_size = sum(tensor_sizes.get(tensor_dict[tensor_type], 0) for tensor_type in tensor_types)
            unique_layer_structures[layer_hash] = (f"layer_{layer_id}", layer_size)
        for tensor_name, tensor_hash in non_layer_tensors.items():
            layer_hash = hashlib.sha256(f"{tensor_name}:{tensor_hash}".encode()).hexdigest()
            unique_layer_structures[layer_hash] = (tensor_name, tensor_sizes.get(tensor_hash, 0))
    total_layers = sum(len(model_layers) + len(non_layer_tensors) for model_layers, non_layer_tensors in all_models_layers)
    unique_layer_count = len(unique_layer_structures)
    layer_sizes = [size for _, size in unique_layer_structures.values()]
    max_layer_size = max(layer_sizes) if layer_sizes else 0
    avg_layer_size = sum(layer_sizes) / len(layer_sizes) if layer_sizes else 0
    print(f"Total layers: {total_layers}")
    print(f"Unique layers: {unique_layer_count}")
    print(f"Unique ratio: {unique_layer_count / total_layers if total_layers else 0:.4f}")
    print(f"Max layer size: {max_layer_size} bytes ({max_layer_size / 1024 / 1024:.2f} MB)")
    print(f"Average layer size: {avg_layer_size:.2f} bytes ({avg_layer_size / 1024 / 1024:.2f} MB)")
    return {
        "total_layers": total_layers,
        "unique_layers": unique_layer_count,
        "unique_ratio": unique_layer_count / total_layers if total_layers else 0,
        "max_layer_size": max_layer_size,
        "avg_layer_size": avg_layer_size
    }

In [ ]:
# Analyze FastCDC chunks
chunk_stats = analyze_fastcdc_chunks(fastcdc_dir)
# Analyze tensor deduplication
tensor_stats = analyze_tensor_dedup(zipllm_dir)
# Analyze layer deduplication
layer_stats = analyze_layer_dedup(zipllm_dir)
# Analyze file deduplication
file_stats = analyze_file_dedup(zipllm_dir)

In [ ]:
from typing import Dict
# ---- Unit helpers ----
PB_BYTES = 1024 ** 5
GB_BYTES = 1024 ** 3
MB_BYTES = 1024 ** 2

PROJECTED_HF_MODEL_SIZE_PB = 17.0   # total HF model size
METADATA_SIZE_BYTES = 64            # per-object metadata size (e.g., hash+overhead)
Experiment_Model_Size_GB = 74.79

def compute_and_print(name: str, stats: Dict[str, Any], unique_key: str, avg_size_key: str):
    """
    Compute:
      - Estimated Metadata (MB): unique_hashes * metadata_size
      - Projected HF Metadata (GB): estimated_metadata_mb * (PROJECTED_HF_MODEL_SIZE_PB * 1024 / Experiment_Model_Size_GB)
    Then print nicely.
    """
    if not stats:
        print(f"[WARN] No stats for {name}, skip.")
        return

    unique_hashes = int(stats.get(unique_key, 0) or 0)
    avg_size_bytes = float(stats.get(avg_size_key, 0.0) or 0.0)

    if unique_hashes <= 0 or avg_size_bytes <= 0:
        print(f"[WARN] Invalid values for {name}: unique={unique_hashes}, avg_size={avg_size_bytes} bytes; skip.")
        return

    # Estimated metadata for the catalog of unique hashes (in MB)
    estimated_metadata_mb = (unique_hashes * METADATA_SIZE_BYTES) / MB_BYTES

    # Projected metadata if scaled to the whole HF corpus (in GB)
    # Formula: estimated_metadata_mb * (PROJECTED_HF_MODEL_SIZE_PB * 1024 / Experiment_Model_Size_GB)
    projected_metadata_gb = estimated_metadata_mb * (PROJECTED_HF_MODEL_SIZE_PB * 1024) / Experiment_Model_Size_GB

    def human(n):
        return f"{n:,.2f}"

    print(f"\n===== {name} =====")
    print(f"Unique Hashes           : {unique_hashes:,}")
    print(f"Avg Size                : {avg_size_bytes/MB_BYTES:.3f} MB")
    print(f"Estimated Metadata (MB) : {human(estimated_metadata_mb)}")
    print(f"Projected HF Metadata (GB): {human(projected_metadata_gb)}")


compute_and_print("ChunkDedup (FastCDC)", chunk_stats,  "unique_chunks",  "avg_chunk_size")
compute_and_print("TensorDedup (ours)",  tensor_stats, "unique_tensors", "avg_tensor_size")
compute_and_print("LayerDedup",           layer_stats,  "unique_layers",  "avg_layer_size")
compute_and_print("FileDedup",            file_stats,   "unique_files",   "avg_file_size")